# Bibliotecas

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import os
from datetime import date, timedelta

# Clases y Funciones propias

In [32]:
class Sismo:
    pass

In [125]:
def get_data(s):
    
    sismos = []
    for i in range(2,len(s)):
        sismo = Sismo()
        sismo.evento = s[i].find_all('td')[0].text[19:] #Lugar
        sismo.fecha_local = s[i].find_all('td')[0].text[:19] #Lugar
        sismo.fecha_utc = s[i].find_all('td')[1].text # Fecha
        sismo.lat = float(s[i].find_all('td')[2].text.split()[0]) #Latitud
        sismo.lng = float(s[i].find_all('td')[2].text.split()[1]) # Longitud
        sismo.profundidad = s[i].find_all('td')[3].text # Profundidad
        sismo.magnitud = s[i].find_all('td')[4].text #Magnitud 
        sismos.append(sismo)
    return pd.DataFrame.from_records([t.__dict__ for t in sismos ])

In [126]:
def get_source(fecha):
    fecha_data = pd.to_datetime(fecha)
    url = 'https://www.sismologia.cl/sismicidad/catalogo/'+fecha_data.strftime('%Y/%m/%Y%m%d')+'.html'
    print("Ruta_log: ",url)
    ssm = requests.get(url)
    s = BeautifulSoup(ssm.text, 'lxml').find_all('tr')
    return s

In [127]:
def process_data(df):
    df[["fecha_local", "hora_local"]] = df["fecha_local"].str.split(' ', expand=True)
    df[["fecha_utc", "hora_utc"]] = df["fecha_utc"].str.split(' ', expand=True)
    df[["profundidad", "profundidad_unidad"]] = df["profundidad"].str.split(' ', expand=True)
    df[["magnitud", "magnitud_unidad"]] = df["magnitud"].str.split(' ', expand=True)
    
    return df[['evento', 'lat', 'lng', 'fecha_local', 'hora_local', 
            'fecha_utc', 'hora_utc', 'profundidad', 'profundidad_unidad','magnitud','magnitud_unidad']]

In [128]:
def get_today_as_string():
    return date.today().strftime("%m/%d/%Y")

In [129]:
def get_yesterday_as_string():
    yesterday = date.today() - timedelta(days=1)
    return yesterday.strftime("%m/%d/%Y")

In [135]:
def save_file(df, fecha):
    fecha_data = pd.to_datetime(fecha)
    nombre_archivo = fecha_data.strftime('%Y%m%d') + '_sismos_' + '.csv' 
    file_dir = os.path.dirname(os.path.abspath("__file__"))
    csv_folder = 'data/' + fecha_data.strftime('%Y') + "/" + fecha_data.strftime('%m')
    try:
        os.makedirs(os.path.join(file_dir, csv_folder))
    except FileExistsError:
        pass
    file_path = os.path.join(file_dir, csv_folder, nombre_archivo)
    df.to_csv(file_path, index=False)
    print('Archivo guardado', nombre_archivo)
    print("")

In [131]:
def process_yesterday():
    #Obtenemos la fecha del día anterior
    yesterday = get_yesterday_as_string()

    #Obtenemos los datos de nuestro sitio
    fuente = get_source(yesterday)

    #Obtenemos los datos en formato dataframe
    df_sismos = get_data(fuente)

    #Procesar datos
    df_sismos = process_data(df_sismos)

    #Guardamos nuestros datos en formato csv
    save_file(df_sismos, yesterday)

In [132]:
def process_history(date):
    #Obtenemos los datos de nuestro sitio
    fuente = get_source(date)

    #Obtenemos los datos en formato dataframe
    df_sismos = get_data(fuente)

    #Procesar datos
    df_sismos = process_data(df_sismos)

    #Guardamos nuestros datos en formato csv
    save_file(df_sismos, date)

In [133]:
def create_process_history(fecha_inicio, sleep_time = 2):
    
    #la historia se mide hasta 2 días atras, el día anterior es parte del scraping diario
    fecha_termino = (date.today() - timedelta(days=2)).strftime("%m/%d/%Y") 

    rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_termino)

    for fecha in rango_fechas:
        process_history(fecha.strftime("%m/%d/%Y"))
        sleep(sleep_time)

# Realizando lectura para ayer
### Esta funcion se deberia aislar para ejecutar en el job que implementaremos en GitHub, para capturar los datos del día anterior
* Opción 1: Generar las llamadas a funciones múltiples funciones

In [95]:
#Obtenemos la fecha del día anterior
yesterday = get_yesterday_as_string()

#Obtenemos los datos de nuestro sitio
fuente = get_source(yesterday)

#Obtenemos los datos en formato dataframe
df_sismos = get_data(fuente)

#Procesar datos
df_sismos = process_data(df_sismos)

#Guardamos nuestros datos en formato csv
save_file(df_sismos, yesterday)

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220820.html
Archivo guardado sismos_20220820.csv


* Opcion 2: Llamar a una funcion que encapsula todo el proceso anterior

In [138]:
#Consumimos una la funcion que hace todo el proceso para el día de ayer
process_yesterday()

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220820.html
Archivo guardado 20220820_sismos_.csv



Como se puede ver, el resultado es el mismo.

# Procesando la historia
### Ahora generaremos un proceso para guardar los datos de la historia que queremos manejar. De esta manera, tendremos un archivo csv por cada día.
**Nota:** Se considera historia hasta 2 días atras del día de la ejecución. El día anterior debe obtenerse con el proceso automático diario.

In [137]:
#Se definen las fechas como texto
fecha_inicio = "08/01/2022" #origen desde agosto 2022
create_process_history(fecha_inicio, 5) #Segundo parámetro es el tiempo de sleep para cada día


Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220801.html
Archivo guardado 20220801_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220802.html
Archivo guardado 20220802_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220803.html
Archivo guardado 20220803_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220804.html
Archivo guardado 20220804_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220805.html
Archivo guardado 20220805_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220806.html
Archivo guardado 20220806_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220807.html
Archivo guardado 20220807_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo/2022/08/20220808.html
Archivo guardado 20220808_sismos_.csv

Ruta_log:  https://www.sismologia.cl/sismicidad/catalogo